# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Data-Import-and-Preprocessing" data-toc-modified-id="Data-Import-and-Preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Import and Preprocessing</a></div><div class="lev2 toc-item"><a href="#Distances" data-toc-modified-id="Distances-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Distances</a></div><div class="lev1 toc-item"><a href="#Matérn-spatio-temporal-kernel" data-toc-modified-id="Matérn-spatio-temporal-kernel-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Matérn spatio-temporal kernel</a></div><div class="lev2 toc-item"><a href="#optimization" data-toc-modified-id="optimization-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>optimization</a></div><div class="lev2 toc-item"><a href="#resulting-kernel" data-toc-modified-id="resulting-kernel-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>resulting kernel</a></div>

Using Matern spatial covariance components instead of RQIso or Squared Exponential.

In [1]:
using TimeSeries
using DataFrames
using GaussianProcesses
using GaussianProcesses: Mean, Kernel, evaluate, metric, IsotropicData, VecF64
using GaussianProcesses: Stationary, KernelData, MatF64
import GaussianProcesses: optimize!, get_optim_target, cov, grad_slice!
import GaussianProcesses: num_params, set_params!, get_params, update_mll!, update_mll_and_dmll!
import GaussianProcesses: get_param_names, cov!, addcov!, multcov!
import Proj4
using Optim
using Distances
;

In [2]:
import PyPlot; plt=PyPlot
using LaTeXStrings
plt.rc("figure", dpi=300.0)
# plt.rc("figure", figsize=(6,4))
plt.rc("savefig", dpi=300.0)
plt.rc("text", usetex=true)
plt.rc("font", family="serif")
plt.rc("font", serif="Palatino")
;

In [3]:
data_dir = "../"

"../"

# Data Import and Preprocessing

In [4]:
include(data_dir*"src/preprocessing.jl")

test_data (generic function with 1 method)

In [5]:
include(data_dir*"src/TempModel.jl")

TempModel

In [6]:
isdList=read_isdList(; data_dir=data_dir)
isdList[1:5,:]

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] anynull(::NullableArrays.NullableArray{Float64,1}) at ./deprecated.jl:57
 [3] convert(::Type{Array{Float64,2}}, ::DataTables.DataTable) at /Users/imolk/Library/Julia/packages/v0.6/DataTables/src/abstractdatatable/abstractdatatable.jl:545
 [4] convert(::Type{Array{T,2} where T}, ::DataTables.DataTable) at /Users/imolk/Library/Julia/packages/v0.6/DataTables/src/abstractdatatable/abstractdatatable.jl:535
 [5] #read_isdList#6(::String, ::Function) at /Users/imolk/Documents/Harvard/Natesh/temperature_model/src/preprocessing.jl:36
 [6] (::#kw##read_isdList)(::Array{Any,1}, ::#read_isdList) at ./<missing>:0
 [7] include_string(::String, ::String) at ./loading.jl:515
 [8] include_string(::Module, ::String, ::String) at /Users/imolk/Library/Julia/packages/v0.6/Compat/src/Compat.jl:464
 [9] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/imolk/Library/Julia/packages/v0.6/IJulia/src/execute_request.jl:154
 [10] eventloo

,USAF,WBAN,NAME,CTRY,STATE,ICAO,LAT,LON,ELEV,BEGIN,END,X_PRJ,Y_PRJ
1,10010,99999,"""JAN MAYEN(NOR-NAVY)""","""NO""",#NULL,"""ENJA""",70.933,-8.667,9.0,1931,2015,4.5545e6,6.11344e6
2,10060,99999,"""EDGEOYA""","""NO""",#NULL,#NULL,78.25,22.817,14.0,1973,2015,4.04982e6,7.5564e6
3,10070,99999,"""NY-ALESUND""","""SV""",#NULL,#NULL,78.917,11.933,7.7,1973,2015,3.8678e6,7.26549e6
4,10080,99999,"""LONGYEAR""","""SV""",#NULL,"""ENSB""",78.246,15.466,26.8,1975,2015,3.99705e6,7.33669e6
5,10090,99999,"""KARL XII OYA""","""SV""",#NULL,#NULL,80.65,25.0,5.0,1955,2015,3.69259e6,7.68545e6


In [7]:
isdSubset=isdList[[(usaf in (725450,725460,725480,725485)) for usaf in isdList[:USAF].values],:]
isdSubset

,USAF,WBAN,NAME,CTRY,STATE,ICAO,LAT,LON,ELEV,BEGIN,END,X_PRJ,Y_PRJ
1,725450,14990,"""THE EASTERN IOWA AIRPORT""","""US""","""IA""","""KCID""",41.883,-91.717,264.6,1973,2015,1.64799e6,1.0441e6
2,725460,14933,"""DES MOINES INTERNATIONAL AIRPORT""","""US""","""IA""","""KDSM""",41.534,-93.653,291.7,1973,2015,1.48723e6,1.00379e6
3,725480,94910,"""WATERLOO MUNICIPAL AIRPORT""","""US""","""IA""","""KALO""",42.554,-92.401,264.6,1960,2015,1.59025e6,1.11766e6
4,725485,14940,"""MASON CITY MUNICIPAL ARPT""","""US""","""IA""","""KMCW""",43.154,-93.327,373.4,1973,2015,1.51407e6,1.18374e6


In [8]:
hourly_cat=read_Stations(isdSubset; data_dir=data_dir)
hourly_cat[1:5,:]

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] convert(::Type{Float64}, ::Base.Dates.Millisecond) at ./deprecated.jl:57
 [3] copy!(::IndexLinear, ::Array{Float64,1}, ::IndexLinear, ::Array{Base.Dates.Millisecond,1}) at ./abstractarray.jl:655
 [4] convert(::Type{Array{Float64,1}}, ::Array{Base.Dates.Millisecond,1}) at ./array.jl:340
 [5] add_ts_hours!(::DataTables.DataTable) at /Users/imolk/Documents/Harvard/Natesh/temperature_model/src/preprocessing.jl:44
 [6] #read_Stations#7(::String, ::Function, ::DataTables.DataTable) at /Users/imolk/Documents/Harvard/Natesh/temperature_model/src/preprocessing.jl:52
 [7] (::#kw##read_Stations)(::Array{Any,1}, ::#read_Stations, ::DataTables.DataTable) at ./<missing>:0
 [8] include_string(::String, ::String) at ./loading.jl:515
 [9] include_string(::Module, ::String, ::String) at /Users/imolk/Library/Julia/packages/v0.6/Compat/src/Compat.jl:464
 [10] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/imolk/Library/Julia/pa

,year,month,day,hour,min,seconds,temp,ts,station,ts_hours
1,2015,1,1,0,52,0,-7.8,2015-01-01T00:52:00,1,0.866667
2,2015,1,1,1,52,0,-8.3,2015-01-01T01:52:00,1,1.86667
3,2015,1,1,2,52,0,-8.3,2015-01-01T02:52:00,1,2.86667
4,2015,1,1,3,52,0,-9.4,2015-01-01T03:52:00,1,3.86667
5,2015,1,1,4,52,0,-9.4,2015-01-01T04:52:00,1,4.86667


## Distances

To get distances between stations, we can either use a function to compute distances on a sphere, or we can first project the coordinates onto a Euclidean plane, and then compute normal distances. I'll do it both ways to check they're consistent (equal up to a multiplication constant), and then use Euclidean distances for convenience.

In [9]:
# http://www.johndcook.com/blog/python_longitude_latitude/
function distance_on_unit_sphere(lat1, long1, lat2, long2)
 
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = π/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.
         
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
     
    cosangle = (sin(phi1)*sin(phi2)*cos(theta1 - theta2) +
           cos(phi1)*cos(phi2))
    arc = acos( cosangle )
 
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc
end

distance_on_unit_sphere (generic function with 1 method)

In [10]:
numstations = DataTables.nrow(isdSubset)
pairwiseSphere = zeros(numstations, numstations)
for i in 1:numstations
    for j in 1:i
        if i==j
            continue
        end
        station1 = isdSubset[i,:]
        station2 = isdSubset[j,:]
        lat1= get(station1[1,:LAT])
        lon1 = get(station1[1,:LON])
        lat2 = get(station2[1,:LAT])
        lon2 = get(station2[1,:LON])
        pairwiseSphere[i,j] = distance_on_unit_sphere(lat1, lon1, lat2, lon2)
        pairwiseSphere[j,i] = pairwiseSphere[i,j]
    end
end
pairwiseSphere

4×4 Array{Float64,2}:
 0.0        0.0259496  0.0146736  0.0303475
 0.0259496  0.0        0.024088   0.0285853
 0.0146736  0.024088   0.0        0.0158124
 0.0303475  0.0285853  0.0158124  0.0      

In [11]:
pairwiseEuclid=pairwise(Euclidean(), Matrix(isdSubset[[:X_PRJ,:Y_PRJ]])')

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] anynull(::NullableArrays.NullableArray{Float64,1}) at ./deprecated.jl:57
 [3] convert(::Type{Array{Float64,2}}, ::DataTables.DataTable) at /Users/imolk/Library/Julia/packages/v0.6/DataTables/src/abstractdatatable/abstractdatatable.jl:545
 [4] convert(::Type{Array{T,2} where T}, ::DataTables.DataTable) at /Users/imolk/Library/Julia/packages/v0.6/DataTables/src/abstractdatatable/abstractdatatable.jl:535
 [5] Array{T,2} where T(::DataTables.DataTable) at ./sysimg.jl:24
 [6] include_string(::String, ::String) at ./loading.jl:515
 [7] include_string(::Module, ::String, ::String) at /Users/imolk/Library/Julia/packages/v0.6/Compat/src/Compat.jl:464
 [8] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/imolk/Library/Julia/packages/v0.6/IJulia/src/execute_request.jl:154
 [9] eventloop(::ZMQ.Socket) at /Users/imolk/Library/Julia/packages/v0.6/IJulia/src/eventloop.jl:8
 [10] (::IJulia.##14#17)() at ./task.jl:335
while lo

4×4 Array{Float64,2}:
      0.0        165736.0        93510.4        1.93474e5
 165736.0             0.0            1.53559e5  1.81942e5
  93510.4             1.53559e5      0.0        1.00846e5
      1.93474e5       1.81942e5      1.00846e5  0.0      

Ratio of the two distance matrices: close enough to a constant!

In [12]:
pairwiseEuclid ./ pairwiseSphere

4×4 Array{Float64,2}:
 NaN            6.38684e6    6.37271e6    6.37527e6
   6.38684e6  NaN            6.37493e6    6.36489e6
   6.37271e6    6.37493e6  NaN            6.37765e6
   6.37527e6    6.36489e6    6.37765e6  NaN        

# Matérn spatio-temporal kernel

In [13]:
mk1 = fix(Periodic(log(0.5), log(√(3.0)), log(24.0)), :lp)
mk2 = Mat32Iso(log(2.0), log(√3.0)) # short term Matern
mk3 = SEIso(log(2.0), log(√3.0)) # short term SE
mk4 = Mat32Iso(log(1.0*24), log(√10.0))
mk5 = RQIso(log(10.0*24), log(√10.0), 3.0)
mk_nonoise=mk1+mk2+mk3+mk4+mk5

temporal_minx = [-1.5563, -0.0368977, 1.06012, -1.38956, -1.19061, -0.0300269, -1.03743, 1.86563, 0.9376, 3.21125, 1.41512, 6.0]
set_params!(mk_nonoise, temporal_minx[2:end])

ksp1 = SEIso(log(2*10^5), log(1.0))
ksp2 = SEIso(log(2*10^5), log(1.0))
ksp3 = SEIso(log(2*10^5), log(1.0))
ksp4 = SEIso(log(2*10^5), log(1.0))
ksp5 = SEIso(log(2*10^5), log(1.0))
k_means = SEIso(log(10^4), log(10.0))

k_spatiotemporal_1 = Masked(mk1, [1]) * Masked(ksp1, [2,3]) +
                     Masked(mk2, [1]) * Masked(ksp2, [2,3]) +
                     Masked(mk3, [1]) * Masked(ksp3, [2,3]) +
                     Masked(mk4, [1]) * Masked(ksp4, [2,3]) +
                     Masked(mk5, [1]) * Masked(ksp5, [2,3]) +
                     fix(Masked(k_means, [2,3]))
logNoise=temporal_minx[1]

-1.5563

## optimization

In [14]:
begin
    k_spatiotemporal = k_spatiotemporal_1
    chunks=GPE[]
    chunk_width=24*10
    tstart=0.0
    tend=tstart+chunk_width
    nobsv=0
    while tstart < get(maximum(hourly_cat[:ts_hours]))
        in_chunk=(tstart .<= hourly_cat[:ts_hours].values) & (hourly_cat[:ts_hours].values .< tend)
        hourly_chunk = hourly_cat[in_chunk,:]
        nobsv_chunk = sum(in_chunk)
        nobsv += nobsv_chunk

        chunk_X_PRJ = isdSubset[:X_PRJ].values[hourly_chunk[:station].values]
        chunk_Y_PRJ = isdSubset[:Y_PRJ].values[hourly_chunk[:station].values]
        chunk_X = [hourly_chunk[:ts_hours].values chunk_X_PRJ chunk_Y_PRJ]

        y = hourly_chunk[:temp].values
        chunk = GPE(chunk_X', y, MeanConst(mean(y)), k_spatiotemporal, logNoise)
        push!(chunks, chunk)

        tstart=tend
        tend+=chunk_width
    end
    reals = TempModel.GPRealisations(chunks)
    @time opt_out=TempModel.optimize_NLopt(reals, mean=false, x_tol=1e-5, f_tol=1e-10)
    @printf(" mLL: %.5f\n", reals.mll)
    println("dmLL: ", reals.dmll)
end

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] &(::BitArray{1}, ::BitArray{1}) at ./deprecated.jl:57
 [3] macro expansion at ./In[14]:9 [inlined]
 [4] anonymous at ./<missing>:?
 [5] include_string(::String, ::String) at ./loading.jl:515
 [6] include_string(::Module, ::String, ::String) at /Users/imolk/Library/Julia/packages/v0.6/Compat/src/Compat.jl:464
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/imolk/Library/Julia/packages/v0.6/IJulia/src/execute_request.jl:154
 [8] eventloop(::ZMQ.Socket) at /Users/imolk/Library/Julia/packages/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##14#17)() at ./task.jl:335
while loading In[14], in expression starting on line 1


5409.872897 seconds (2.33 M allocations: 919.166 GiB, 7.37% gc time)
 mLL: -46015.85174
dmLL: [8.37715e-5, -0.0017365, 0.00176894, 7.55825e-6, 0.00176894, -0.00167956, 0.000586397, -0.000297559, 0.000586397, 0.000311219, 0.00067744, -0.00053022, 0.00067744, 0.00184674, -0.000578759, 0.00151395, -0.000578759, -0.000317266, 0.00100581, -6.57274, -0.00103178, 0.00100581]


## resulting kernel

In [21]:
print("mk₁: Periodic \n=================\n")
@printf("σ: %5.3f\n", √mk1.kern.σ2)
@printf("l: %5.3f\n", √mk1.kern.ℓ2)
@printf("p: %5.0f hours\n", mk1.kern.p)
print("> spatial decay:\n")
@printf("l: %5.3f km\n", √ksp1.ℓ2 / 1000)

print("\nmk₂: Matérn \n=================\n")
@printf("σ: %5.3f\n", √ mk2.σ2)
@printf("l: %5.3f hours\n", mk2.ℓ)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp2.σ2)
@printf("l: %5.3f km\n", √ksp2.ℓ2 / 1000)

print("\nmk₃: SEIso \n=================\n")
@printf("σ: %5.3f\n", √mk3.σ2)
@printf("l: %5.3f hours\n", √mk3.ℓ2)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp3.σ2)
@printf("l: %5.3f km\n", √ksp3.ℓ2 / 1000)

print("\nmk₄: Matérn \n=================\n")
@printf("σ: %5.3f\n", √mk4.σ2)
@printf("l: %5.3f hours\n", mk4.ℓ)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp4.σ2)
@printf("l: %5.3f km \n", √ksp4.ℓ2 / 1000)

print("\nmk₅: RQIso \n=================\n")
@printf("σ: %5.3f\n", √mk5.σ2)
@printf("l: %5.3f days\n", √mk5.ℓ2 / 24)
@printf("α: %5.3f\n",  mk5.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp5.σ2)
@printf("l: %5.3f km\n", √ksp5.ℓ2 / 1000)

mk₁: Periodic 
σ: 2.761
l: 0.786
p:    24 hours
> spatial decay:
l: 798.133 km

mk₂: Matérn 
σ: 0.308
l: 0.278 hours
> spatial decay:
l: 42.836 km

mk₃: SEIso 
σ: 0.644
l: 3.026 hours
> spatial decay:
l: 155.575 km

mk₄: Matérn 
σ: 1.383
l: 1.841 hours
> spatial decay:
l: 17.906 km 

mk₅: RQIso 
σ: 4.048
l: 0.368 days
α: 20.086
> spatial decay:
l: 464.523 km
